In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

In [26]:
import pandas as pd

# Load datasets
matches = pd.read_csv('./Dataset/Downloaded/matches.csv')
home_away = pd.read_csv('./Dataset/Downloaded/teamwise_home_and_away.csv')
teams = pd.read_csv('./Dataset/Downloaded/teams.csv')

# Quick preview
matches.head()


,id,Season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,IPL-2017,Hyderabad,05-04-2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,IPL-2017,Pune,06-04-2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,IPL-2017,Rajkot,07-04-2017,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,IPL-2017,Indore,08-04-2017,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,IPL-2017,Bangalore,08-04-2017,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [27]:
matches.isnull().sum()

id                   0
Season               0
city                 7
date                 0
team1                0
team2                0
toss_winner          0
toss_decision        0
result               0
dl_applied           0
winner               4
win_by_runs          0
win_by_wickets       0
player_of_match      4
venue                0
umpire1              2
umpire2              2
umpire3            637
dtype: int64

In [28]:
home_away.isnull().sum()

team                   0
home_wins              0
away_wins              0
home_matches           0
away_matches           0
home_win_percentage    0
away_win_percentage    0
dtype: int64

In [29]:
teams

,team1
0,Pune Warriors
1,Kolkata Knight Riders
2,Rajasthan Royals
3,Kochi Tuskers Kerala
4,Gujarat Lions
5,Chennai Super Kings
6,Rising Pune Supergiants
7,Delhi Daredevils
8,Deccan Chargers
9,Delhi Capitals


In [30]:
# Fill missing 'city' with 'venue' (optional)
matches['city'].fillna(matches['venue'], inplace=True)

# Fill minor umpire nulls (optional)
matches['umpire1'].fillna('Unknown', inplace=True)
matches['umpire2'].fillna('Unknown', inplace=True)

# Drop 'umpire3' since it's not consistently used
matches.drop(columns=['umpire3'], inplace=True)

# No need to drop rows with null 'winner' or 'player_of_match'
# These usually indicate "No result", rain-affected matches etc.


C:\Users\GC\AppData\Local\Temp\ipykernel_19776\422207515.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\GC\AppData\Local\Temp\ipykernel_19776\422207515.py:5: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [31]:
#1.Most wins by team

import plotly.express as px

team_wins = matches['winner'].value_counts().reset_index()
team_wins.columns = ['Team', 'Wins']

fig = px.bar(team_wins, x='Team', y='Wins', color='Wins',
             title='🏆 Most Wins by Team in IPL',
             text='Wins')
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Team', yaxis_title='Total Wins')
fig.show()


In [32]:
#2.Home vs Away Performance

home_away['home_win_percent'] = (home_away['home_wins'] / home_away['home_matches']) * 100
home_away['away_win_percent'] = (home_away['away_wins'] / home_away['away_matches']) * 100

# Melt for grouped bar chart
ha_melted = pd.melt(home_away, id_vars='team',
                    value_vars=['home_win_percent', 'away_win_percent'],
                    var_name='Location', value_name='Win %')
ha_melted['Location'] = ha_melted['Location'].map({'home_win_percent': 'Home', 'away_win_percent': 'Away'})

fig = px.bar(ha_melted, x='team', y='Win %', color='Location',
             barmode='group', title='Home vs Away Win % per Team')
fig.update_layout(xaxis_title='Team', yaxis_title='Win Percentage')
fig.show()


In [33]:
#3. Toss Decision Impact

toss_result = matches.copy()
toss_result['toss_win_match_win'] = toss_result['toss_winner'] == toss_result['winner']

toss_summary = toss_result.groupby('toss_decision')['toss_win_match_win'].mean().reset_index()
toss_summary['Win % After Toss'] = toss_summary['toss_win_match_win'] * 100

fig = px.bar(toss_summary, x='toss_decision', y='Win % After Toss', color='Win % After Toss',
             title='🏏 Toss Decision vs Match Win %',
             text='Win % After Toss')
fig.update_traces(textposition='outside')
fig.update_layout(xaxis_title='Toss Decision', yaxis_title='Win %')
fig.show()


In [34]:
#4. Team Performance by Season

season_wins = matches.groupby(['Season', 'winner']).size().reset_index(name='Wins')

fig = px.bar(season_wins, x='Season', y='Wins', color='winner',
             title='Team Wins by Season', text='Wins')
fig.update_layout(xaxis_title='Season', yaxis_title='Wins', legend_title='Team')
fig.show()


In [35]:
sorted(matches.columns)

['Season',
 'city',
 'date',
 'dl_applied',
 'id',
 'player_of_match',
 'result',
 'team1',
 'team2',
 'toss_decision',
 'toss_winner',
 'umpire1',
 'umpire2',
 'venue',
 'win_by_runs',
 'win_by_wickets',
 'winner']

In [36]:
# Player Analysis Roadmap

In [37]:
import pandas as pd

deliveries = pd.read_csv("./Dataset/Downloaded/deliveries.csv")
deliveries.head()


,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [38]:
deliveries.isnull().sum()

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
is_super_over            0
wide_runs                0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
player_dismissed    170244
dismissal_kind      170244
fielder             172630
dtype: int64

In [39]:
deliveries.shape

(179078, 21)

In [40]:
#1. Top Run Scorers

top_scorers = deliveries.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(10).reset_index()

import plotly.express as px
fig = px.bar(top_scorers, x='batsman', y='batsman_runs', color='batsman_runs',
             title='🏏 Top 10 Run Scorers in IPL', text='batsman_runs')
fig.update_layout(xaxis_title='Player', yaxis_title='Runs')
fig.show()


In [41]:
#2. Best Strike Rates (min 200 balls)

# Get total runs and balls for each batsman
runs_scored = deliveries.groupby('batsman')['batsman_runs'].sum()
balls_faced = deliveries.groupby('batsman').size()

# Combine both into one DataFrame
batting_stats = pd.DataFrame({
    'runs': runs_scored,
    'balls': balls_faced
})

# Filter players with at least 200 balls faced
batting_stats = batting_stats[batting_stats['balls'] >= 200]

# Calculate strike rate
batting_stats['strike_rate'] = (batting_stats['runs'] / batting_stats['balls']) * 100

# Sort and select top 10
top_strike_rates = batting_stats.sort_values(by='strike_rate', ascending=False).head(10).reset_index()

# Plot using Plotly
import plotly.express as px

fig = px.bar(top_strike_rates, x='batsman', y='strike_rate', color='strike_rate',
             title='🚀 Best Strike Rates (Min 200 Balls Faced)', text='strike_rate')
fig.update_layout(xaxis_title='Player', yaxis_title='Strike Rate')
fig.show()


In [42]:
#3. Top Wicket Takers

# Remove NaNs in 'player_dismissed' → only rows where wicket fell
wickets = deliveries[deliveries['player_dismissed'].notnull()]

top_wicket_takers = wickets['bowler'].value_counts().head(10).reset_index()
top_wicket_takers.columns = ['Bowler', 'Wickets']

fig = px.bar(top_wicket_takers, x='Bowler', y='Wickets', color='Wickets',
             title='🎯 Top 10 Wicket Takers in IPL', text='Wickets')
fig.update_layout(xaxis_title='Bowler', yaxis_title='Wickets')
fig.show()


In [43]:
# 4. Top 6s and 4s Hitters

# 6s
sixes = deliveries[deliveries['batsman_runs'] == 6]['batsman'].value_counts().head(10).reset_index()
sixes.columns = ['Batsman', 'Sixes']

fig = px.bar(sixes, x='Batsman', y='Sixes', color='Sixes', title='💣 Most Sixes Hit', text='Sixes')
fig.update_layout(xaxis_title='Player', yaxis_title='Sixes')
fig.show()

# 4s
fours = deliveries[deliveries['batsman_runs'] == 4]['batsman'].value_counts().head(10).reset_index()
fours.columns = ['Batsman', 'Fours']

fig = px.bar(fours, x='Batsman', y='Fours', color='Fours', title='🏏 Most Fours Hit', text='Fours')
fig.update_layout(xaxis_title='Player', yaxis_title='Fours')
fig.show()


In [44]:
#Bonus: Compare Two Players Side by Side

def compare_batters(player1, player2):
    stats = {}
    for player in [player1, player2]:
        df = deliveries[deliveries['batsman'] == player]
        runs = df['batsman_runs'].sum()
        balls = df.shape[0]
        strike_rate = (runs / balls) * 100 if balls > 0 else 0
        fours = df[df['batsman_runs'] == 4].shape[0]
        sixes = df[df['batsman_runs'] == 6].shape[0]
        stats[player] = {
            'Runs': runs,
            'Balls': balls,
            'SR': round(strike_rate, 2),
            '4s': fours,
            '6s': sixes
        }

    return pd.DataFrame(stats)

compare_batters('V Kohli', 'RG Sharma')


,V Kohli,RG Sharma
Runs,5434.00,4914.00
Balls,4211.00,3816.00
SR,129.04,128.77
4s,482.00,431.00
6s,191.00,194.00


HTML plots for hovering 

In [45]:
# all plots in single html file

import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import io
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.graph_objs import Figure
import os


In [46]:

# --- Plot 1: Top run scorers ---
fig1= px.bar(top_scorers, x='batsman', y='batsman_runs', color='batsman_runs',
             title='🏏 Top 10 Run Scorers in IPL', text='batsman_runs')
fig1.update_layout(xaxis_title='Player', yaxis_title='Runs')


# --- Plot 2: Best Strike Rates (Min 200 Balls Faced)---
fig2= px.bar(top_strike_rates, x='batsman', y='strike_rate', color='strike_rate',
             title='🚀 Best Strike Rates (Min 200 Balls Faced)', text='strike_rate')
fig2.update_layout(xaxis_title='Player', yaxis_title='Strike Rate')

# --- Plot 3: Top 10 Wicket Takers in IPL ---
fig3= px.bar(top_wicket_takers, x='Bowler', y='Wickets', color='Wickets',
             title='🎯 Top 10 Wicket Takers in IPL', text='Wickets')
fig3.update_layout(xaxis_title='Bowler', yaxis_title='Wickets')


# --- Plot 4: Most Sixes Hitters ---
fig4= px.bar(sixes, x='Batsman', y='Sixes', color='Sixes', title='💣 Most Sixes Hit', text='Sixes')
fig4.update_layout(xaxis_title='Player', yaxis_title='Sixes')


# Save all plots in a single HTML file
#html_path = "Dataset/netflix_analysis_all_plots.html"

#pio.write_html([fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9], file=html_path, auto_open=False)

#html_path


In [47]:
import plotly.io as pio

# List of all your figures
figures = [fig1, fig2, fig3, fig4]

# Generate HTML strings for each figure
html_parts = [pio.to_html(fig, full_html=False, include_plotlyjs='cdn') for fig in figures]

# Combine them into one full HTML document
full_html = f"""
<html>
<head>
    <title>IPL Cricket Data Analysis</title>
</head>
<body>
    {''.join(html_parts)}
</body>
</html>
"""

# Save to file
output_path = "IPL_Cricket_Data_Analysis_all_plots.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"Saved to {output_path}")


Saved to IPL_Cricket_Data_Analysis_all_plots.html
